In [37]:
import os
from pathlib import Path

In [38]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_chroma import Chroma
# # libmagic fix https://github.com/zooniverse/panoptes-python-client/issues/264 #issuecomment-891725587

In [44]:
LANGCHAIN_TRACING_V2='true'
LANGCHAIN_API_KEY='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'

In [39]:
output_dir = Path(r"D:\Ed\RAGs\Streamlit_RAG_1\docling\Manual fr RAG")
loader = DirectoryLoader(output_dir, glob="*.md")
docs = loader.load()
len(docs)
from langsmith import traceable
from langchain_community.embeddings import GPT4AllEmbeddings
# # Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
all_splits_pypdf = text_splitter.split_documents(docs)
all_splits_pypdf_texts = [d.page_content for d in all_splits_pypdf]
baseline = Chroma.from_texts(
    texts=all_splits_pypdf_texts,
    collection_name="baseline",
    embedding=GPT4AllEmbeddings(),
)
retriever_baseline = baseline.as_retriever()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [40]:
from langchain_ollama import ChatOllama, OllamaLLM

llm_llama3 = ChatOllama(
    model="llama3.2:1b",
    temperature=0,
    # other params...
)

In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm_llama3,
    chain_type="stuff",
    retriever=retriever_baseline
)

In [42]:
qa.invoke('what is the accuracy of CI1A J Thermocouple?')

{'query': 'what is the accuracy of CI1A J Thermocouple?',
 'result': "According to the text, the accuracy of CI1A J Thermocouple is:\n\n* 0 to 115°C (32 to 240°F) larger of -2% or -3°C (-6°F)\n* 116 to 225°C (241 to 440°F) larger of -5% or -6°C (-10°F)\n\nSo, the accuracy range for CI1A J Thermocouple is:\n\n* -2% or -3°C (-6°F) from 0 to 115°C (32 to 240°F)\n* -5% or -6°C (-10°F) from 116 to 225°C (241 to 440°F)\n\nI don't know the accuracy of CI1A J Thermocouple, I don't know."}

In [43]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.invoke,
        description=(
            'use this tool when answering queries to get '
            'more information about the topic'
        )
    )
]

In [27]:
from langchain.agents import initialize_agent

In [44]:
# @traceable
agent = initialize_agent(
    agent='chat-conversational-react-description',
    # agent='chat-zero-shot-react-description',
    tools=tools,
    llm=llm_llama3,
    verbose=True,
    max_iterations=5,
    early_stopping_method='generate',
    memory=conversational_memory,
    return_intermediate_steps = True,
    handle_parsing_errors=True
)

In [35]:
agent

AgentExecutor(memory=ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'), verbose=True, tags=['chat-zero-shot-react-description'], agent=ChatAgent(llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nKnowledge Base: use this tool when answering queries to get more information about the topic\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: Knowledge Base\n\nThe $JSON_BLOB should on

In [45]:
agent("what is the accuracy of CI1A J Thermocouple?")



> Entering new AgentExecutor chain...
{
    "action": "Knowledge Base",
    "action_input": ""
}
 

Please provide more information about the Thermocouple you are referring to. I will do my best to assist you.

Also, please let me know which type of response you would like me to provide:

> Knowledge Base
> Final Answer

ValueError: text must not be None or empty

In [54]:
from langchain.agents import tool

In [57]:
memory_key = "history"

In [59]:
from langchain.memory import ConversationBufferMemory

In [61]:
memory = ConversationBufferMemory(memory_key=memory_key, return_messages=True)

C:\Users\GopiP\AppData\Local\Temp\ipykernel_27592\805325693.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key=memory_key, return_messages=True)


In [56]:
@tool
def tool(query):
    "Searches and returns documents regarding the llm powered autonomous agents blog"
    retriever_baseline.invoke(query)
    return docs

In [64]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema import SystemMessage
from langchain.prompts import MessagesPlaceholder

In [63]:
system_message = SystemMessage(
        content=(
            "Do your best to answer the questions. "
            "Feel free to use any tools available to look up "
            "relevant information, only if neccessary"
        )
)

In [ ]:
prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
    )

In [65]:
agent = OpenAIFunctionsAgent(llm=llm_llama3, tools=tools, prompt=prompt)

C:\Users\GopiP\AppData\Local\Temp\ipykernel_27592\4276050586.py:1: LangChainDeprecationWarning: The class `OpenAIFunctionsAgent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~create_openai_functions_agent` instead.
  agent = OpenAIFunctionsAgent(llm=llm_llama3, tools=tools, prompt=prompt)


In [66]:
from langchain.agents import AgentExecutor

In [70]:
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True, handle_parsing_errors=True)

In [71]:
agent_executor({"input": "what is the accuracy of CI1A J Thermocouple?"})



> Entering new AgentExecutor chain...


KeyError: 'tool_names'

In [37]:
from langchain.agents import tool_calling_agent 

In [38]:
tool_calling_agent

<module 'langchain.agents.tool_calling_agent' from 'd:\\Ed\\RAGs\\fortive_rag_v1\\Multimodal_RAG\\.venv\\Lib\\site-packages\\langchain\\agents\\tool_calling_agent\\__init__.py'>

In [39]:
# Get the prompt to use - you can modify this!
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
prompt = hub.pull("hwchase17/react")

d:\Ed\RAGs\fortive_rag_v1\Multimodal_RAG\.venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [40]:
agent = create_react_agent(llm_llama3, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [42]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Thought: LangChain is a library for building conversational AI models in Python, specifically designed for use with the Transformers framework. It provides an easy-to-use interface for defining and training conversational models using a simple and intuitive API.

Action: 

*   Check if LangChain is installed
*   Import the necessary libraries (e.g., `transformers`)
*   Load a pre-trained model or create a new one from scratch
`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [41]:
agent_executor.invoke({"input","what is the accuracy of CI1A J Thermocouple?"})

IndexError: list index out of range

In [78]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAI

In [79]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

d:\Ed\RAGs\fortive_rag_v1\Multimodal_RAG\.venv\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [87]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [89]:
mdfd_prompt = '''Answer the following questions as best you can. Use only the tools. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final or partial answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

In [90]:
prompt.template = mdfd_prompt

In [92]:
# Create an agent executor by passing in the agent and tools
agent = create_react_agent(llm_llama3, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [93]:
agent_executor.invoke({"input": "what is the accuracy of CI1A J Thermocouple?"})



> Entering new AgentExecutor chain...
I'll do my best to provide a helpful response.

Action: I will search for information about CI1A J Thermocouple.
Invalid Format: Missing 'Action Input:' after 'Action:'Question: what is the accuracy of CI1A J Thermocouple?

Knowledge Base: The CI1A J Thermocouple, also known as the CI1A thermocouple, is a type of thermocouple used in various applications. However, without more specific information about its accuracy, it's challenging to provide an accurate answer.

Action Input: I will search for information about the accuracy of CI1A J Thermocouple.
Invalid Format: Missing 'Action:' after 'Thought:'Question: what is the accuracy of CI1A J Thermocouple?

Knowledge Base: The CI1A J Thermocouple, also known as the CI1A thermocouple, is a type of thermocouple used in various applications. However, without more specific information about its accuracy, it's challenging to provide an accurate answer.

Action Input: I will search for information about t

ResponseError: an error was encountered while running the model: read tcp 127.0.0.1:59817->127.0.0.1:59748: wsarecv: An existing connection was forcibly closed by the remote host.